In [1]:
import torch
import optuna
# import comet_ml
# import wandb

In [2]:
wandb.login()

[02/26/24 17:05:07] ERROR    Failed to detect the name of this notebook, you can set it manually     ]8;id=170980;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/wandb/jupyter.py\jupyter.py]8;;\:]8;id=463580;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/wandb/jupyter.py#231\231]8;;\
                             with the WANDB_NOTEBOOK_NAME environment variable to enable code                      
                             saving.                                                                               

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:wandb: Appending key for api.wandb.ai to your netrc file: /home/rahit/.netrc


True

In [3]:
import random

# start a new wandb run to track this script
wandb.init(
    # set the wandb project where this run will be logged
    project="testing-wandb-from-graham",
    
    # track hyperparameters and run metadata
    config={
    "learning_rate": 0.02,
    "architecture": "CNN",
    "dataset": "CIFAR-100",
    "epochs": 10,
    }
)

# simulate training
epochs = 10
offset = random.random() / 5
for epoch in range(2, epochs):
    acc = 1 - 2 ** -epoch - random.random() / epoch - offset
    loss = 2 ** -epoch + random.random() / epoch + offset
    
    # log metrics to wandb
    wandb.log({"acc": acc, "loss": loss})
    
# [optional] finish the wandb run, necessary in notebooks
wandb.finish()

wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


wandb: Network error (ConnectTimeout), entering retry loop.
wandb: Network error (ConnectTimeout), entering retry loop.


Problem at: /tmp/ipykernel_14626/2810626247.py 4 <module>


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│    1 import random                                                                               │
│    2                                                                                             │
│    3 # start a new wandb run to track this script                                                │
│ ❱  4 wandb.init(                                                                                 │
│    5 │   # set the wandb project where this run will be logged                                   │
│    6 │   project="testing-wandb-from-graham",                                                    │
│    7                                                                                             │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/wandb/sdk/wandb_init.py:1195 in init         │
│                                                                                                  │
│   1192 │   except Error as e:                                                                    │
│   1193 │   │   if logger is not None:                                                            │
│   1194 │   │   │   logger.exception(str(e))                                                      │
│ ❱ 1195 │   │   raise e                                                                           │
│   1196 │   except KeyboardInterrupt as e:                                                        │
│   1197 │   │   assert logger                                                                     │
│   1198 │   │   logger.warning("interrupted", exc_info=e)                                         │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/wandb/sdk/wandb_init.py:1176 in init         │
│                                                                                                  │
│   1173 │   │   assert wi.settings                                                                │
│   1174 │   │   except_exit = wi.settings._except_exit                                            │
│   1175 │   │   try:                                                                              │
│ ❱ 1176 │   │   │   run = wi.init()                                                               │
│   1177 │   │   │   except_exit = wi.settings._except_exit                                        │
│   1178 │   │   except (KeyboardInterrupt, Exception) as e:                                       │
│   1179 │   │   │   if not isinstance(e, KeyboardInterrupt):                                      │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/wandb/sdk/wandb_init.py:785 in init          │
│                                                                                                  │
│    782 │   │   │   │   # we don't need to do console cleanup at this point                       │
│    783 │   │   │   │   backend.cleanup()                                                         │
│    784 │   │   │   │   self.teardown()                                                           │
│ ❱  785 │   │   │   raise error                                                                   │
│    786 │   │                                                                                     │
│    787 │   │   assert run_result is not None  # for mypy                                         │
│    788                                                     

[02/26/24 17:10:27] WARNING  Retrying (Retry(total=2, connect=None, read=None, redirect=None, ]8;id=100601;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=740605;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py#812\812]8;;\
                             status=None)) after connection broken by                                              
                             'NewConnectionError('<urllib3.connection.HTTPSConnection object                       
                             at 0x2ad71dff3460>: Failed to establish a new connection: [Errno                      
                             110] Connection timed out')': /api/4504800232407040/store/                            

[02/26/24 17:12:34] WARNING  Retrying (Retry(total=1, connect=None, read=None, redirect=None, ]8;id=108339;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=860122;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py#812\812]8;;\
                             status=None)) after connection broken by                                              
                             'NewConnectionError('<urllib3.connection.HTTPSConnection object                       
                             at 0x2ad71df66b20>: Failed to establish a new connection: [Errno                      
                             110] Connection timed out')': /api/4504800232407040/store/                            

[02/26/24 17:14:41] WARNING  Retrying (Retry(total=0, connect=None, read=None, redirect=None, ]8;id=896227;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=315876;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py#812\812]8;;\
                             status=None)) after connection broken by                                              
                             'NewConnectionError('<urllib3.connection.HTTPSConnection object                       
                             at 0x2ad71dff31f0>: Failed to establish a new connection: [Errno                      
                             110] Connection timed out')': /api/4504800232407040/store/                            

[02/26/24 17:18:56] WARNING  Retrying (Retry(total=2, connect=None, read=None, redirect=None, ]8;id=478572;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=333945;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py#812\812]8;;\
                             status=None)) after connection broken by                                              
                             'NewConnectionError('<urllib3.connection.HTTPSConnection object                       
                             at 0x2ad71f4e80a0>: Failed to establish a new connection: [Errno                      
                             110] Connection timed out')': /api/4504800232407040/envelope/                         

[02/26/24 17:21:03] WARNING  Retrying (Retry(total=1, connect=None, read=None, redirect=None, ]8;id=615600;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=592642;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py#812\812]8;;\
                             status=None)) after connection broken by                                              
                             'NewConnectionError('<urllib3.connection.HTTPSConnection object                       
                             at 0x2ad71d72b700>: Failed to establish a new connection: [Errno                      
                             110] Connection timed out')': /api/4504800232407040/envelope/                         

[02/26/24 17:23:11] WARNING  Retrying (Retry(total=0, connect=None, read=None, redirect=None, ]8;id=857925;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py\connectionpool.py]8;;\:]8;id=649869;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/urllib3/connectionpool.py#812\812]8;;\
                             status=None)) after connection broken by                                              
                             'NewConnectionError('<urllib3.connection.HTTPSConnection object                       
                             at 0x2ad71df66ca0>: Failed to establish a new connection: [Errno                      
                             110] Connection timed out')': /api/4504800232407040/envelope/                         

In [2]:
data = torch.load('./data/05_model_input/2024-02-monarch_heterodata_v1.pt', map_location='cpu')

In [3]:
print(f""" 
Total nodes: {data.num_nodes}
Total node types: {len(data.node_types)}

Total edges: {data.num_edges}
Total edge types: {len(data.edge_types)}                
""")

 
Total nodes: 862115
Total node types: 88

Total edges: 11412471
Total edge types: 289                



In [4]:
data.has_isolated_nodes()

True

In [5]:
data.is_undirected()

False

In [28]:
list(sorted([(v.edge_index.shape[1],key) for (key,v) in data.edge_items()], reverse=True))

[(4096780, ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene')),
 (1472390,
  ('biolink:Gene',
   'biolink:expressed_in',
   'biolink:GrossAnatomicalStructure')),
 (883206,
  ('biolink:Gene', 'biolink:has_phenotype', 'biolink:PhenotypicFeature')),
 (841329,
  ('biolink:Gene', 'biolink:enables', 'biolink:BiologicalProcessOrActivity')),
 (747796,
  ('biolink:Gene',
   'biolink:actively_involved_in',
   'biolink:BiologicalProcessOrActivity')),
 (551383, ('biolink:Gene', 'biolink:orthologous_to', 'biolink:Gene')),
 (502219, ('biolink:Gene', 'biolink:located_in', 'biolink:CellularComponent')),
 (387549,
  ('biolink:Gene', 'biolink:expressed_in', 'biolink:AnatomicalEntity')),
 (241100,
  ('biolink:Disease', 'biolink:has_phenotype', 'biolink:PhenotypicFeature')),
 (227356,
  ('biolink:PhenotypicFeature',
   'biolink:subclass_of',
   'biolink:PhenotypicFeature')),
 (198370, ('biolink:Gene', 'biolink:participates_in', 'biolink:Pathway')),
 (168548,
  ('biolink:Gene',
   'biolink:acts_ups

In [22]:
sorted_etype = []
for k, e in data.edge_items():
    print(k)
    print(e[1].edge_index.shape[1])

('biolink:Gene', 'biolink:orthologous_to', 'biolink:Gene')


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 sorted_etype = []                                                                            │
│   2 for k, e in data.edge_items():                                                               │
│   3 │   print(k)                                                                                 │
│ ❱ 4 │   print(e[1].edge_index.shape[1])                                                          │
│   5                                                                                              │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch_geometric/data/storage.py:118 in       │
│ __getitem__                                                                                      │
│                                                                                                  │
│   115 │   │   │   del self[key]                                                                  │
│   116 │                                                                                          │
│   117 │   def __getitem__(self, key: str) -> Any:                                                │
│ ❱ 118 │   │   return self._mapping[key]                                                          │
│   119 │                                                                                          │
│   120 │   def __setitem__(self, key: str, value: Any) -> None:                                   │
│   121 │   │   self._pop_cache(key)                                                               │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
KeyError: 1

In [6]:

zip((data.edge_types, for n in data.edge))

[
    ('biolink:Gene', 'biolink:orthologous_to', 'biolink:Gene'),
    ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'),
    ('biolink:Gene', 'biolink:actively_involved_in', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:Gene', 'biolink:acts_upstream_of_or_within', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:Pathway', 'biolink:related_to', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:BiologicalProcessOrActivity', 'biolink:subclass_of', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:Gene', 'biolink:acts_upstream_of', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:BiologicalProcessOrActivity', 'biolink:related_to', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:Gene', 'biolink:has_phenotype', 'biolink:PhenotypicFeature'),
    ('biolink:Disease', 'biolink:has_phenotype', 'biolink:PhenotypicFeature'),
    ('biolink:PhenotypicFeature', 'biolink:subclass_of', 'biolink:PhenotypicFeature'),
    ('biolink:Gene', 'biolink:participates_in', 'biolink:Pathway'),
    ('biolink:MolecularEntity', 'biolink:participates_in', 'biolink:Pathway'),
    ('biolink:Gene', 'biolink:enables', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:Gene', 'biolink:contributes_to', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:PhenotypicFeature', 'biolink:related_to', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:Gene', 'biolink:active_in', 'biolink:CellularComponent'),
    ('biolink:Gene', 'biolink:located_in', 'biolink:CellularComponent'),
    ('biolink:Gene', 'biolink:expressed_in', 'biolink:CellularComponent'),
    ('biolink:Gene', 'biolink:colocalizes_with', 'biolink:CellularComponent'),
    ('biolink:MacromolecularComplex', 'biolink:related_to', 'biolink:CellularComponent'),
    ('biolink:CellularComponent', 'biolink:related_to', 'biolink:CellularComponent'),
    ('biolink:CellularComponent', 'biolink:subclass_of', 'biolink:CellularComponent'),
    ('biolink:BiologicalProcessOrActivity', 'biolink:related_to', 'biolink:CellularComponent'),
    ('biolink:Gene', 'biolink:part_of', 'biolink:MacromolecularComplex'),
    ('biolink:Gene', 'biolink:expressed_in', 'biolink:MacromolecularComplex'),
    ('biolink:Disease', 'biolink:related_to', 'biolink:PhenotypicFeature'),
    ('biolink:Gene', 'biolink:expressed_in', 'biolink:GrossAnatomicalStructure'),
    ('biolink:PhenotypicFeature', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'),
    ('biolink:GrossAnatomicalStructure', 'biolink:subclass_of', 'biolink:GrossAnatomicalStructure'),
    ('biolink:GrossAnatomicalStructure', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'),
    ('biolink:Disease', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'),
    ('biolink:AnatomicalEntity', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'),
    ('biolink:BiologicalProcessOrActivity', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'),
    ('biolink:Gene', 'biolink:expressed_in', 'biolink:AnatomicalEntity'),
    ('biolink:AnatomicalEntity', 'biolink:subclass_of', 'biolink:AnatomicalEntity'),
    ('biolink:Gene', 'biolink:acts_upstream_of_or_within_positive_effect', 'biolink:BiologicalProcessOrActivity'),
    ('biolink:PhenotypicFeature', 'biolink:related_to', 'biolink:CellularComponent'),
    ('biolink:Disease', 'biolink:related_to', 'biolink:CellularComponent'),
    ('biolink:Cell', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'),
    ('biolink:GrossAnatomicalStructure', 'biolink:related_to', 'biolink:AnatomicalEntity'),
    ('biolink:PhenotypicFeature', 'biolink:related_to', 'biolink:AnatomicalEntity'),
    ('biolink:AnatomicalEntity', 'biolink:related_to', 'biolink:AnatomicalEntity'),
    ('biolink:Gene', 'biolink:gene_associated_with_condition', 'biolink:Disease'),
    ('biolink:Disease', 'biolink:subclass_of', 'biolink:Disease'),
    ('biolink:Gene', 'biolink:causes', 'biolink:Disease'),
    ('biolink:Gene', 'biolink:contributes_to', 'biolink:Disease'),
    ('biolink:Protein', 'biolink:participate

In [ ]:
data.metapath_dict

In [23]:
print(data.edge_index_dict.keys())

dict_keys([('biolink:Gene', 'biolink:orthologous_to', 'biolink:Gene'), ('biolink:Gene', 'biolink:interacts_with', 
'biolink:Gene'), ('biolink:Gene', 'biolink:actively_involved_in', 'biolink:BiologicalProcessOrActivity'), 
('biolink:Gene', 'biolink:acts_upstream_of_or_within', 'biolink:BiologicalProcessOrActivity'), ('biolink:Pathway', 
'biolink:related_to', 'biolink:BiologicalProcessOrActivity'), ('biolink:BiologicalProcessOrActivity', 
'biolink:subclass_of', 'biolink:BiologicalProcessOrActivity'), ('biolink:Gene', 'biolink:acts_upstream_of', 
'biolink:BiologicalProcessOrActivity'), ('biolink:BiologicalProcessOrActivity', 'biolink:related_to', 
'biolink:BiologicalProcessOrActivity'), ('biolink:Gene', 'biolink:has_phenotype', 'biolink:PhenotypicFeature'), 
('biolink:Disease', 'biolink:has_phenotype', 'biolink:PhenotypicFeature'), ('biolink:PhenotypicFeature', 
'biolink:subclass_of', 'biolink:PhenotypicFeature'), ('biolink:Gene', 'biolink:participates_in', 
'biolink:Pathway'), ('biolink:MolecularEntity', 'biolink:participates_in', 'biolink:Pathway'), ('biolink:Gene', 
'biolink:enables', 'biolink:BiologicalProcessOrActivity'), ('biolink:Gene', 'biolink:contributes_to', 
'biolink:BiologicalProcessOrActivity'), ('biolink:PhenotypicFeature', 'biolink:related_to', 
'biolink:BiologicalProcessOrActivity'), ('biolink:Gene', 'biolink:active_in', 'biolink:CellularComponent'), 
('biolink:Gene', 'biolink:located_in', 'biolink:CellularComponent'), ('biolink:Gene', 'biolink:expressed_in', 
'biolink:CellularComponent'), ('biolink:Gene', 'biolink:colocalizes_with', 'biolink:CellularComponent'), 
('biolink:MacromolecularComplex', 'biolink:related_to', 'biolink:CellularComponent'), ('biolink:CellularComponent',
'biolink:related_to', 'biolink:CellularComponent'), ('biolink:CellularComponent', 'biolink:subclass_of', 
'biolink:CellularComponent'), ('biolink:BiologicalProcessOrActivity', 'biolink:related_to', 
'biolink:CellularComponent'), ('biolink:Gene', 'biolink:part_of', 'biolink:MacromolecularComplex'), 
('biolink:Gene', 'biolink:expressed_in', 'biolink:MacromolecularComplex'), ('biolink:Disease', 
'biolink:related_to', 'biolink:PhenotypicFeature'), ('biolink:Gene', 'biolink:expressed_in', 
'biolink:GrossAnatomicalStructure'), ('biolink:PhenotypicFeature', 'biolink:related_to', 
'biolink:GrossAnatomicalStructure'), ('biolink:GrossAnatomicalStructure', 'biolink:subclass_of', 
'biolink:GrossAnatomicalStructure'), ('biolink:GrossAnatomicalStructure', 'biolink:related_to', 
'biolink:GrossAnatomicalStructure'), ('biolink:Disease', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'),
('biolink:AnatomicalEntity', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'), 
('biolink:BiologicalProcessOrActivity', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'), ('biolink:Gene',
'biolink:expressed_in', 'biolink:AnatomicalEntity'), ('biolink:AnatomicalEntity', 'biolink:subclass_of', 
'biolink:AnatomicalEntity'), ('biolink:Gene', 'biolink:acts_upstream_of_or_within_positive_effect', 
'biolink:BiologicalProcessOrActivity'), ('biolink:PhenotypicFeature', 'biolink:related_to', 
'biolink:CellularComponent'), ('biolink:Disease', 'biolink:related_to', 'biolink:CellularComponent'), 
('biolink:Cell', 'biolink:related_to', 'biolink:GrossAnatomicalStructure'), ('biolink:GrossAnatomicalStructure', 
'biolink:related_to', 'biolink:AnatomicalEntity'), ('biolink:PhenotypicFeature', 'biolink:related_to', 
'biolink:AnatomicalEntity'), ('biolink:AnatomicalEntity', 'biolink:related_to', 'biolink:AnatomicalEntity'), 
('biolink:Gene', 'biolink:gene_associated_with_condition', 'biolink:Disease'), ('biolink:Disease', 
'biolink:subclass_of', 'biolink:Disease'), ('biolink:Gene', 'biolink:causes', 'biolink:Disease'), ('biolink:Gene', 
'biolink:contributes_to', 'biolink:Disease'), ('biolink:Protein', 'biolink:participates_in', 'biolink:Pathway'), 
('biolink:ChemicalEntity', 'biolink:participates_in', 'biolink:Pathway'), ('biolink:Gene', 
'biolink:acts_upstream_of_posi

In [11]:
# list(mapped_nodes.keys())
# specify the metapath schemas as a list of lists of node types.
metapaths = [
    ['biolink:Gene', "biolink:MolecularEntity", 'biolink:Gene'],
    ['biolink:Gene', "biolink:SmallMolecule", 'biolink:Gene'],
    ['biolink:Gene', "biolink:ChemicalEntity", 'biolink:Gene'],
    ['biolink:Gene', "biolink:Cell", 'biolink:Gene'],
    ['biolink:Gene', "biolink:PhenotypicFeature", 'biolink:Gene'],
    ['biolink:Gene', "biolink:BiologicalProcessOrActivity", 'biolink:Gene'],
    ['biolink:Gene', "biolink:CellularComponent", 'biolink:Gene'],
    ['biolink:Gene', "biolink:LifeStage", 'biolink:Gene'],
    ['biolink:Gene', "biolink:Disease", 'biolink:Gene'],
    ['biolink:Gene', "biolink:PhenotypicQuality", 'biolink:Gene'],
    ['biolink:Gene', "biolink:CellularOrganism", 'biolink:Gene'],
    ['biolink:Gene', "biolink:PathologicalProcess", 'biolink:Gene'],
    ['biolink:Gene', "biolink:Pathway", 'biolink:Gene'],
    ['biolink:Gene', "biolink:Pathway", 'biolink:Gene'],
]


In [19]:
# list(mapped_nodes.keys())
# specify the metapath schemas as a list of lists of node types.
metapaths = [
    ('biolink:Gene', "biolink:MolecularEntity", 'biolink:Gene'),
    ('biolink:Gene', "biolink:SmallMolecule", 'biolink:Gene'),
    ('biolink:Gene', "biolink:ChemicalEntity", 'biolink:Gene'),
    ('biolink:Gene', "biolink:Cell", 'biolink:Gene'),
    ('biolink:Gene', "biolink:PhenotypicFeature", 'biolink:Gene'),
    ('biolink:Gene', "biolink:BiologicalProcessOrActivity", 'biolink:Gene'),
    ('biolink:Gene', "biolink:CellularComponent", 'biolink:Gene'),
    ('biolink:Gene', "biolink:LifeStage", 'biolink:Gene'),
    ('biolink:Gene', "biolink:Disease", 'biolink:Gene'),
    ('biolink:Gene', "biolink:PhenotypicQuality", 'biolink:Gene'),
    ('biolink:Gene', "biolink:CellularOrganism", 'biolink:Gene'),
    ('biolink:Gene', "biolink:PathologicalProcess", 'biolink:Gene'),
    ('biolink:Gene', "biolink:Pathway", 'biolink:Gene'),
    ('biolink:Gene', "biolink:Pathway", 'biolink:Gene'),
]


In [8]:
metapaths = [
    ('biolink:Gene', 'biolink:orthologous_to', 'biolink:Gene'),
    ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'),
    # ('biolink:Gene', 'biolink:actively_involved_in', 'biolink:BiologicalProcessOrActivity'),
    # ('biolink:Gene', 'biolink:acts_upstream_of_or_within', 'biolink:BiologicalProcessOrActivity'),
    # ('biolink:Gene', 'biolink:participates_in', 'biolink:Pathway'),
    # ('biolink:Pathway', 'biolink:related_to', 'biolink:BiologicalProcessOrActivity'),
    # ('biolink:Gene', 'biolink:has_phenotype', 'biolink:PhenotypicFeature'),
    # ('biolink:Disease', 'biolink:has_phenotype', 'biolink:PhenotypicFeature'),
    # ('biolink:Gene', 'biolink:gene_associated_with_condition', 'biolink:Disease'),
    # ('biolink:Gene', 'biolink:contributes_to', 'biolink:Disease'),
    # ('biolink:Gene', 'biolink:active_in', 'biolink:CellularComponent'),
    # ('biolink:Gene', 'biolink:expressed_in', 'biolink:CellularComponent'),
    # ('biolink:Gene', 'biolink:expressed_in', 'biolink:AnatomicalEntity'),
    # ('biolink:PhenotypicFeature', 'biolink:related_to', 'biolink:AnatomicalEntity'),
    # ('biolink:Gene', 'biolink:actively_involved_in', 'biolink:Pathway'),
    # ('biolink:MolecularEntity', 'biolink:participates_in', 'biolink:Pathway'),
    # ('biolink:Gene', 'biolink:acts_upstream_of', 'biolink:BiologicalProcessOrActivity'),
    # ('biolink:Gene', 'biolink:enables', 'biolink:BiologicalProcessOrActivity'),
    # ('biolink:Disease', 'biolink:related_to', 'biolink:BiologicalProcessOrActivity'),
    # ('biolink:Disease', 'biolink:related_to', 'biolink:PhenotypicFeature')
]


In [31]:
# 1. Must be same node for end and begining of consicutive metapath
# 2. Must be circular
metapaths = [
    ('biolink:Gene', 'biolink:orthologous_to', 'biolink:Gene'),
    ('biolink:Gene', 'biolink:interacts_with', 'biolink:Gene'),
    ('biolink:Gene', 'biolink:expressed_in', 'biolink:GrossAnatomicalStructure')
]

# Since Monarch architecture does not have circular relationship (from paper we know) we will have to calculate embedding for different metapaths and then compress it together with another layer to get final aggregated embedding

In [ ]:
data.get_edge_index(['biolink:Gene', "biolink:Pathway", 'biolink:Gene'])

In [ ]:
import os.path as osp
import sys

import matplotlib.pyplot as plt
import torch
from sklearn.manifold import TSNE

from torch_geometric.datasets import Planetoid
from torch_geometric.nn import Node2Vec

# path = osp.join(osp.dirname(osp.realpath(__file__)), '..', 'data', 'Planetoid')
# dataset = Planetoid(path, name='Cora')
# data = dataset[0]

device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Node2Vec(
    data['biolink:MolecularEntity'].edge_index,
    embedding_dim=128,
    walk_length=20,
    context_size=10,
    walks_per_node=10,
    num_negative_samples=1,
    p=1.0,
    q=1.0,
    sparse=True,
).to(device)

num_workers = 4 if sys.platform == 'linux' else 0
loader = model.loader(batch_size=128, shuffle=True, num_workers=num_workers)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train():
    model.train()
    total_loss = 0
    for pos_rw, neg_rw in loader:
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss / len(loader)


# @torch.no_grad()
# def test():
#     model.eval()
#     z = model()
#     acc = model.test(
#         train_z=z[data.train_mask],
#         train_y=data.y[data.train_mask],
#         test_z=z[data.test_mask],
#         test_y=data.y[data.test_mask],
#         max_iter=150,
#     )
#     return acc


for epoch in range(1, 101):
    loss = train()
    # acc = test()
    print(f'Epoch: {epoch:03d}, Loss: {loss:.4f}, Acc: --')


# @torch.no_grad()
# def plot_points(colors):
#     model.eval()
#     z = model().cpu().numpy()
#     z = TSNE(n_components=2).fit_transform(z)
#     y = data.y.cpu().numpy()

#     plt.figure(figsize=(8, 8))
#     for i in range(dataset.num_classes):
#         plt.scatter(z[y == i, 0], z[y == i, 1], s=20, color=colors[i])
#     plt.axis('off')
#     plt.show()


# colors = [
#     '#ffc0cb', '#bada55', '#008080', '#420420', '#7fe5f0', '#065535', '#ffd700'
# ]
# plot_points(colors)

In [6]:
torch.cuda.is_available()

True

In [15]:
torch.cuda.empty_cache()

In [7]:
!nvidia-smi

Mon Feb 26 04:05:13 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla P100-PCIE-12GB           Off | 00000000:04:00.0 Off |                    0 |
| N/A   38C    P0              27W / 250W |      2MiB / 12288MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [8]:
!export 'PYTORCH_CUDA_ALLOC_CONF=max_split_size_mb:512'

In [9]:
import ray

In [30]:
from torch_geometric.nn import MetaPath2Vec


device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,
                     metapath=metapaths, walk_length=50, context_size=7,
                     walks_per_node=5, num_negative_samples=5,
                     sparse=True).to(device)

loader = model.loader(batch_size=128, shuffle=True, num_workers=1)
optimizer = torch.optim.SparseAdam(list(model.parameters()), lr=0.01)


def train(epoch, log_steps=100, eval_steps=2000):
    model.train()

    total_loss = 0
    for i, (pos_rw, neg_rw) in enumerate(loader):
        optimizer.zero_grad()
        loss = model.loss(pos_rw.to(device), neg_rw.to(device))
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
        if (i + 1) % log_steps == 0:
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Loss: {total_loss / log_steps:.4f}'))
            total_loss = 0

        if (i + 1) % eval_steps == 0:
            acc = test()
            print((f'Epoch: {epoch}, Step: {i + 1:05d}/{len(loader)}, '
                   f'Acc: {acc:.4f}'))


# @torch.no_grad()
# def test(train_ratio=0.1):
#     model.eval()

#     z = model('author', batch=data['author'].y_index.to(device))
#     y = data['author'].y

#     perm = torch.randperm(z.size(0))
#     train_perm = perm[:int(z.size(0) * train_ratio)]
#     test_perm = perm[int(z.size(0) * train_ratio):]

#     return model.test(z[train_perm], y[train_perm], z[test_perm], y[test_perm],
#                       max_iter=150)

from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score

@torch.no_grad()
def test(n_clusters=10):
    """
    Test the learned embeddings by performing K-Means clustering and computing the Silhouette Score.
    
    Args:
    - embeddings (Tensor): The node embeddings learned by MetaPath2Vec, assumed to be a PyTorch tensor.
    - n_clusters (int): The number of clusters to use in K-Means.
    
    Returns:
    - silhouette_score (float): The Silhouette Score of the clustering, ranging from -1 to 1.
    """
    # Setting the mode to evaluation
    model.eval()
    embeddings = model('biolink:Gene')
    
    # Move embeddings to CPU and convert to NumPy for compatibility with scikit-learn
    embeddings_np = embeddings.cpu().detach().numpy()
    
    # Perform K-Means clustering
    kmeans = KMeans(n_clusters=n_clusters, random_state=42)
    cluster_labels = kmeans.fit_predict(embeddings_np)
    
    # Calculate Silhouette Score
    silhouette_avg = silhouette_score(embeddings_np, cluster_labels)
    
    print(f"Silhouette Score with {n_clusters} clusters: {silhouette_avg:.4f}")
    return silhouette_avg

accurace_trace = []

for epoch in range(1, 6):
    train(epoch)
    acc = test()
    accurace_trace.append(acc)
    print(f'Epoch: {epoch}, Accuracy: {acc:.4f}')

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:5                                                                                    │
│                                                                                                  │
│    2                                                                                             │
│    3                                                                                             │
│    4 device = 'cuda' if torch.cuda.is_available() else 'cpu'                                     │
│ ❱  5 model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128,                               │
│    6 │   │   │   │   │    metapath=metapaths, walk_length=50, context_size=7,                    │
│    7 │   │   │   │   │    walks_per_node=5, num_negative_samples=5,                              │
│    8 │   │   │   │   │    sparse=True).to(device)                                                │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch_geometric/nn/models/metapath2vec.py:94 │
│ in __init__                                                                                      │
│                                                                                                  │
│    91 │   │                                                                                      │
│    92 │   │   assert walk_length + 1 >= context_size                                             │
│    93 │   │   if walk_length > len(metapath) and metapath[0][0] != metapath[-1][-1]:             │
│ ❱  94 │   │   │   raise AttributeError(                                                          │
│    95 │   │   │   │   "The 'walk_length' is longer than the given 'metapath', but "              │
│    96 │   │   │   │   "the 'metapath' does not denote a cycle")                                  │
│    97                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: The 'walk_length' is longer than the given 'metapath', but the 'metapath' does not denote a cycle

In [14]:
# torch.save(model, './data/06_models/monarch_metapath2vec_v2.pt')

In [5]:
model = torch.load('./data/06_models/monarch_metapath2vec_v1.pt', torch.device('cpu'))

In [6]:
embeddings = model('biolink:Gene')
# Move embeddings to CPU and convert to NumPy for compatibility with scikit-learn
embeddings_np = embeddings.cpu().detach().numpy()

In [18]:
label = list(data.node_types)

In [14]:
data['biolink:Disease']

{'num_nodes': 26940, 'x': tensor([[1.],
        [1.],
        [1.],
        ...,
        [1.],
        [1.],
        [1.]])}

In [ ]:
test()

In [4]:
# from tqdm import tqdm

embeddings = {}
node_types = set(data.node_types)  # Get unique node types from your graph data

for node_type in node_types:
    try:
        embeddings[node_type] = model(node_type).detach().cpu().numpy()  # Extract embeddings for each node type and move to CPU
    except KeyError:
        print(f"Warning: Embeddings for node type '{node_type}' not found. Skipping.")


In [22]:
embeddings.keys()

dict_keys(['biolink:Gene'])

In [6]:
import os
 
cpuCount = os.cpu_count()
print("Number of CPUs in the system:", cpuCount)

Number of CPUs in the system: 44


In [ ]:
import umap

embeddings_2d = {}  # To store 2D projections of embeddings

for node_type, emb in embeddings.items():
    reducer = umap.UMAP(random_state=42)
    embeddings_2d[node_type] = reducer.fit_transform(emb)  # Apply UMAP to each type's embeddings

In [7]:
from sklearn.manifold import TSNE
# from openTSNE import TSNE
# check cuML's UMAP projection

embeddings_2d = {}  # To store 2D projections of embeddings

for node_type, emb in embeddings.items():
    tsne = TSNE(n_components=2, random_state=42, n_jobs=32, verbose=True)
    embeddings_2d[node_type] = tsne.fit_transform(emb)  # Apply t-SNE to each type's embeddings


--------------------------------------------------------------------------------
TSNE(early_exaggeration=12, n_components=3, n_jobs=32, random_state=42,
     verbose=True)
--------------------------------------------------------------------------------
===> Finding 90 nearest neighbors using Annoy approximate search using euclidean distance...


╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│   5                                                                                              │
│   6 for node_type, emb in embeddings.items():                                                    │
│   7 │   tsne = TSNE(n_components=3, random_state=42, n_jobs=32, verbose=True)                    │
│ ❱ 8 │   embeddings_2d[node_type] = tsne.fit(emb)  # Apply t-SNE to each type's embeddings        │
│   9                                                                                              │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/openTSNE/tsne.py:1246 in fit                 │
│                                                                                                  │
│   1243 │   │   if self.verbose:                                                                  │
│   1244 │   │   │   print("-" * 80, repr(self), "-" * 80, sep="\n")                               │
│   1245 │   │                                                                                     │
│ ❱ 1246 │   │   embedding = self.prepare_initial(X, affinities, initialization)                   │
│   1247 │   │                                                                                     │
│   1248 │   │   try:                                                                              │
│   1249 │   │   │   # Early exaggeration with lower momentum to allow points to find more         │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/openTSNE/tsne.py:1324 in prepare_initial     │
│                                                                                                  │
│   1321 │   │   # If precomputed affinites are given, use those, otherwise proceed with           │
│   1322 │   │   # standard perpelxity-based affinites                                             │
│   1323 │   │   if affinities is None:                                                            │
│ ❱ 1324 │   │   │   affinities = MultiscaleMixture(                                               │
│   1325 │   │   │   │   X,                                                                        │
│   1326 │   │   │   │   self.perplexity,                                                          │
│   1327 │   │   │   │   method=self.neighbors,                                                    │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/openTSNE/affinity.py:810 in __init__         │
│                                                                                                  │
│    807 │   │   │   effective_perplexities = self.check_perplexities(perplexities, n_samples)     │
│    808 │   │   │   log.info("KNN index provided. Ignoring KNN-related parameters.")              │
│    809 │   │                                                                                     │
│ ❱  810 │   │   self.__neighbors, self.__distances = self.knn_index.build()                       │
│    811 │   │                                                                                     │
│    812 │   │   with utils.Timer("Calculating affinity matrix...", verbose):                      │
│    813 │   │   │   self.P = self._calculate_P(                                                   │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/openTSN

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(10, 8))

for node_type, emb in embeddings_2d.items():
    plt.scatter(emb[:, 0], emb[:, 1], label=node_type, alpha=0.7, edgecolors='w', s=60)  # Plot each type's embeddings

plt.legend()
plt.title('Node Embeddings Visualization by Type')
plt.xlabel('Component 1')
plt.ylabel('Component 2')
plt.show()


In [16]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'
model = Node2Vec(data['biolink:MolecularEntity'].edge_index, embedding_dim=128, 
            walk_length=20,                        # lenght of rw
            context_size=10, walks_per_node=20,
            num_negative_samples=1, 
            p=200, q=1,                             # bias parameters
            sparse=True).to(device)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:2                                                                                    │
│                                                                                                  │
│   1 device = 'cuda' if torch.cuda.is_available() else 'cpu'                                      │
│ ❱ 2 model = Node2Vec(data['biolink:MolecularEntity'].edge_index, embedding_dim=128,              │
│   3 │   │   │   walk_length=20,                        # lenght of rw                            │
│   4 │   │   │   context_size=10, walks_per_node=20,                                              │
│   5 │   │   │   num_negative_samples=1,                                                          │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch_geometric/data/storage.py:96 in        │
│ __getattr__                                                                                      │
│                                                                                                  │
│    93 │   │   try:                                                                               │
│    94 │   │   │   return self[key]                                                               │
│    95 │   │   except KeyError:                                                                   │
│ ❱  96 │   │   │   raise AttributeError(                                                          │
│    97 │   │   │   │   f"'{self.__class__.__name__}' object has no attribute '{key}'"             │
│    98 │   │   │   ) from None                                                                    │
│    99                                                                                            │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'NodeStorage' object has no attribute 'edge_index'

In [12]:
model = MetaPath2Vec(data.edge_index_dict, embedding_dim=128, metapath=metapaths, walk_length=50, context_size=7, walks_per_node=10, num_negative_samples=5)

optimizer = torch.optim.Adam(model.parameters(), lr=0.01)

def train(num_epochs):
    model.train()
    total_loss = 0
    for _ in range(num_epochs):
        optimizer.zero_grad()
        pos_rw = model.positive_sample(data.edge_index_dict)
        neg_rw = model.negative_sample(pos_rw)
        loss = model.loss(pos_rw, neg_rw)
        loss.backward()
        optimizer.step()
        total_loss += loss.item()
    return total_loss

total_loss = train(1000)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:18                                                                                   │
│                                                                                                  │
│   15 │   │   total_loss += loss.item()                                                           │
│   16 │   return total_loss                                                                       │
│   17                                                                                             │
│ ❱ 18 total_loss = train(1000)                                                                    │
│   19                                                                                             │
│                                                                                                  │
│ in train:10                                                                                      │
│                                                                                                  │
│    7 │   total_loss = 0                                                                          │
│    8 │   for _ in range(num_epochs):                                                             │
│    9 │   │   optimizer.zero_grad()                                                               │
│ ❱ 10 │   │   pos_rw = model.positive_sample(data.edge_index_dict)                                │
│   11 │   │   neg_rw = model.negative_sample(pos_rw)                                              │
│   12 │   │   loss = model.loss(pos_rw, neg_rw)                                                   │
│   13 │   │   loss.backward()                                                                     │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch/nn/modules/module.py:1269 in           │
│ __getattr__                                                                                      │
│                                                                                                  │
│   1266 │   │   │   modules = self.__dict__['_modules']                                           │
│   1267 │   │   │   if name in modules:                                                           │
│   1268 │   │   │   │   return modules[name]                                                      │
│ ❱ 1269 │   │   raise AttributeError("'{}' object has no attribute '{}'".format(                  │
│   1270 │   │   │   type(self).__name__, name))                                                   │
│   1271 │                                                                                         │
│   1272 │   def __setattr__(self, name: str, value: Union[Tensor, 'Module']) -> None:             │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'MetaPath2Vec' object has no attribute 'positive_sample'

In [32]:
import torch_geometric.transforms as T

transform = T.AddRandomWalkPE(walk_length=4, attr_name='pe')
post_data = transform(data)

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:4                                                                                    │
│                                                                                                  │
│   1 import torch_geometric.transforms as T                                                       │
│   2                                                                                              │
│   3 transform = T.AddRandomWalkPE(walk_length=4, attr_name='pe')                                 │
│ ❱ 4 post_data = transform(data)                                                                  │
│   5                                                                                              │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch_geometric/transforms/base_transform.py │
│ :32 in __call__                                                                                  │
│                                                                                                  │
│   29 │   """                                                                                     │
│   30 │   def __call__(self, data: Any) -> Any:                                                   │
│   31 │   │   # Shallow-copy the data so that we prevent in-place data modification.              │
│ ❱ 32 │   │   return self.forward(copy.copy(data))                                                │
│   33 │                                                                                           │
│   34 │   def forward(self, data: Any) -> Any:                                                    │
│   35 │   │   pass                                                                                │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch_geometric/transforms/add_positional_en │
│ coding.py:140 in forward                                                                         │
│                                                                                                  │
│   137 │   │   self.attr_name = attr_name                                                         │
│   138 │                                                                                          │
│   139 │   def forward(self, data: Data) -> Data:                                                 │
│ ❱ 140 │   │   assert data.edge_index is not None                                                 │
│   141 │   │   row, col = data.edge_index                                                         │
│   142 │   │   N = data.num_nodes                                                                 │
│   143 │   │   assert N is not None                                                               │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch_geometric/data/hetero_data.py:159 in   │
│ __getattr__                                                                                      │
│                                                                                                  │
│    156 │   │   │   return getattr(self._global_store, key)                                       │
│    157 │   │   elif bool(re.search('_dict$', key)):                                              │
│    158 │   │   │   return self.collect(key[:-5])                                                 │
│ ❱  159 │   │   raise AttributeError(f"'{self.__class__.__name__}' has no "                       │
│    160 │   │   │   │   │   │   │    f"attribute '{key}'")                                        │
│    161 │                                                   

In [31]:
data.edge_index

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:1                                                                                    │
│                                                                                                  │
│ ❱ 1 data.edge_index                                                                              │
│   2                                                                                              │
│                                                                                                  │
│ /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch_geometric/data/hetero_data.py:159 in   │
│ __getattr__                                                                                      │
│                                                                                                  │
│    156 │   │   │   return getattr(self._global_store, key)                                       │
│    157 │   │   elif bool(re.search('_dict$', key)):                                              │
│    158 │   │   │   return self.collect(key[:-5])                                                 │
│ ❱  159 │   │   raise AttributeError(f"'{self.__class__.__name__}' has no "                       │
│    160 │   │   │   │   │   │   │    f"attribute '{key}'")                                        │
│    161 │                                                                                         │
│    162 │   def __setattr__(self, key: str, value: Any):                                          │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
AttributeError: 'HeteroData' has no attribute 'edge_index'

In [71]:
data.validate()

[02/23/24 04:40:52] WARNING  /home/rahit/jupyter_py3/lib/python3.8/site-packages/torch_geometric/da ]8;id=299331;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/warnings.py\warnings.py]8;;\:]8;id=172803;file:///cvmfs/soft.computecanada.ca/easybuild/software/2020/avx2/Core/python/3.8.10/lib/python3.8/warnings.py#109\109]8;;\
                             ta/data.py:1038: UserWarning: The node types {'biolink:Activity',                     
                             'biolink:NoncodingRNAProduct', 'biolink:RegulatoryRegion',                            
                             'biolink:GeographicExposure', 'biolink:Haplotype',                                    
                             'biolink:CodingSequence', 'biolink:DatasetDistribution',                              
                             'biolink:WebPage', 'biolink:Invertebrate', 'biolink:Attribute',                       
                             'biolink:ProteinDomain', 'biolink:BiologicalSex', 'biolink:Dataset',                  
                             'biolink:Snv', 'biolink:Mammal', 'biolink:OrganismalEntity',                          
                             'biolink:ChemicalMixture', 'biolink:Patent',                                          
                             'biolink:EnvironmentalProcess', 'biolink:PhenotypicSex',                              
                             'biolink:Procedure', 'biolink:Fungus', 'biolink:Treatment',                           
                             'biolink:Human', 'biolink:ProteinFamily', 'biolink:ConfidenceLevel',                  
                             'biolink:Study', 'biolink:Exon', 'biolink:Genotype',                                  
                             'biolink:MaterialSample', 'biolink:DiagnosticAid', 'biolink:Article',                 
                             'biolink:Bacterium', 'biolink:Event', 'biolink:SiRNA',                                
                             'biolink:GeneticInheritance', 'biolink:Agent', 'biolink:CellLine',                    
                             'biolink:ProcessedMaterial', 'biolink:Genome',                                        
                             'biolink:StudyVariable', 'biolink:ReagentTargetedGene',                               
                             'biolink:MicroRNA', 'biolink:Publication',                                            
                             'biolink:TranscriptionFactorBindingSite',                                             
                             'biolink:IndividualOrganism', 'biolink:Zygosity',                                     
                             'biolink:AccessibleDnaRegion', 'biolink:GenotypicSex',                                
                             'biolink:BiologicalEntity', 'biolink:PopulationOfIndividualOrganisms',                
                             'biolink:ChemicalExposure'} are isolated and are not referenced by any                
                             edge type                                                                             
                               warnings.warn(msg)                                                                  
                                                                                                                   

True

I tried to get random walk from the converted networkx graph. But do not have time to finish it rn.

In [6]:
from torch_geometric.utils import to_networkx
import networkx as nx

# Convert to a NetworkX graph for easier manipulation
G = to_networkx(data, to_multi=True)

In [9]:

print(f""" 
Total nodes: {G.number_of_nodes()}
Total edges: {G.number_of_edges()}           
""")

Total nodes: 862115
Total edges: 11412471

In [12]:
G[185]

AdjacencyView({1242: {0: {'type': ('biolink:PhenotypicFeature', 'biolink:subclass_of', 'biolink:PhenotypicFeature')}}, 160687: {0: {'type': ('biolink:PhenotypicFeature', 'biolink:related_to', 'biolink:BiologicalProcessOrActivity')}}})

In [27]:
for a in G[185].items():
    print(a[0])
    print(a[1][0]['type'])

1242

('biolink:PhenotypicFeature', 'biolink:subclass_of', 'biolink:PhenotypicFeature')

160687

('biolink:PhenotypicFeature', 'biolink:related_to', 'biolink:BiologicalProcessOrActivity')

In [ ]:
G['biolink:Gene'].neighbours

In [ ]:
import random

def random_walk_hetero(graph, start_node, walk_length, node_type, edge_type):
    # Initialize the walk
    walk = [start_node]
    
    # Perform the walk
    for _ in range(walk_length):
        current_node = walk[-1]
        for n, nbrsdict in G.adjacency():
            for nbr, keydict in nbrsdict.items():
                for key, eattr in keydict.items():
                    if "weight" in eattr:
                        # Do something useful with the edges
                        pass
        neighbors = [n for n in G[current_node] if n[1][0]['type'] == edge_type]
        if neighbors:
            walk.append(random.choice(neighbors))
        else:
            break  # No more neighbors of the desired type

    return walk

# Example usage
start_node = 0  # Assuming this is a valid node ID in your graph
walk_length = 10
node_type = 'biolink:Gene'
edge_type = 'edge_type_1'
walk = random_walk_hetero(G, start_node, walk_length, node_type, edge_type)
print(walk)


In [17]:
cluster.close()

[02/22/24 23:46:44] INFO     Scheduler closing...                                                 ]8;id=295869;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=433701;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3949\3949]8;;\

                    INFO     Scheduler closing all comms                                          ]8;id=274560;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py\scheduler.py]8;;\:]8;id=69727;file:///home/rahit/jupyter_py3/lib/python3.8/site-packages/distributed/scheduler.py#3971\3971]8;;\

In [18]:
client.close()